In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bank-customer-churn-prediction/Churn_Modelling.csv


# Data Preprocessing

## Importing the dataset

In [2]:
df=pd.read_csv(os.path.join(dirname, filename))

In [3]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,3,15619304,Onio,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,4,15701354,Boni,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43.0,2,125510.82,1,NaN,1.0,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9997,9998,15584532,Liu,709,France,Female,36.0,7,0.00,1,0.0,1.0,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
9999,9999,15682355,Sabbatini,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
10000,10000,15628319,Walker,792,France,Female,28.0,4,130142.79,1,1.0,0.0,38190.78,0


In [4]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          1
Gender             0
Age                1
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          1
IsActiveMember     1
EstimatedSalary    0
Exited             0
dtype: int64

In [6]:
from sklearn.impute import SimpleImputer

# Create imputers
imputer_categorical = SimpleImputer(strategy='most_frequent')
imputer_numeric = SimpleImputer(strategy='median')

df['Geography'] = imputer_categorical.fit_transform(df[['Geography']]).ravel()
df['HasCrCard'] = imputer_categorical.fit_transform(df[['HasCrCard']]).ravel()
df['IsActiveMember'] = imputer_categorical.fit_transform(df[['IsActiveMember']]).ravel()
df['Age'] = imputer_numeric.fit_transform(df[['Age']])

In [7]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [8]:
X=df.iloc[:,3:-1].values
y=df.iloc[:,-1].values

In [9]:
print(X)

[[619 'France' 'Female' ... 1.0 1.0 101348.88]
 [608 'Spain' 'Female' ... 0.0 1.0 112542.58]
 [502 'France' 'Female' ... 1.0 0.0 113931.57]
 ...
 [772 'Germany' 'Male' ... 1.0 0.0 92888.52]
 [792 'France' 'Female' ... 1.0 0.0 38190.78]
 [792 'France' 'Female' ... 1.0 0.0 38190.78]]


In [10]:
print(y)

[1 0 1 ... 1 0 0]


# Encoding categorical data

## Label Encoding the Gender Column

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [12]:
print(X)

[[619 'France' 0 ... 1.0 1.0 101348.88]
 [608 'Spain' 0 ... 0.0 1.0 112542.58]
 [502 'France' 0 ... 1.0 0.0 113931.57]
 ...
 [772 'Germany' 1 ... 1.0 0.0 92888.52]
 [792 'France' 0 ... 1.0 0.0 38190.78]
 [792 'France' 0 ... 1.0 0.0 38190.78]]


## Label Encoding the Geography Column

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [14]:
print(X)

[[1.0 0.0 0.0 ... 1.0 1.0 101348.88]
 [0.0 0.0 1.0 ... 0.0 1.0 112542.58]
 [1.0 0.0 0.0 ... 1.0 0.0 113931.57]
 ...
 [0.0 1.0 0.0 ... 1.0 0.0 92888.52]
 [1.0 0.0 0.0 ... 1.0 0.0 38190.78]
 [1.0 0.0 0.0 ... 1.0 0.0 38190.78]]


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [16]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Building the ANN

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

ann= Sequential()
ann.add(Dense(units=6,activation='relu'))
ann.add(Dense(units=6,activation='relu'))
ann.add(Dense(units=1,activation='sigmoid'))

# Training the ANN

In [18]:
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [19]:
ann.fit(X_train,y_train, batch_size=32,epochs=100)

Epoch 1/100
251/251 [==============================] - 2s 2ms/step - loss: 0.5972 - accuracy: 0.7318
Epoch 2/100
251/251 [==============================] - 0s 2ms/step - loss: 0.4813 - accuracy: 0.7937
Epoch 3/100
251/251 [==============================] - 0s 2ms/step - loss: 0.4560 - accuracy: 0.7937
Epoch 4/100
251/251 [==============================] - 0s 2ms/step - loss: 0.4397 - accuracy: 0.7962
Epoch 5/100
251/251 [==============================] - 0s 2ms/step - loss: 0.4270 - accuracy: 0.8130
Epoch 6/100
251/251 [==============================] - 0s 2ms/step - loss: 0.4163 - accuracy: 0.8201
Epoch 7/100
251/251 [==============================] - 0s 2ms/step - loss: 0.4061 - accuracy: 0.8249
Epoch 8/100
251/251 [==============================] - 0s 2ms/step - loss: 0.3933 - accuracy: 0.8345
Epoch 9/100
251/251 [==============================] - 0s 2ms/step - loss: 0.3779 - accuracy: 0.8439
Epoch 10/100
251/251 [==============================] - 0s 2ms/step - loss: 0.3662 - accura

# Part 4 - Making the predictions and evaluating the model

## Predicting the Test set results

In [20]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 1]]


## Making the Confusion Matrix

In [21]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1555   59]
 [ 197  190]]


0.872063968015992